In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
cabbage_df = pd.read_csv("data-files/merged_df_1.csv")

In [13]:
cabbage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20591 entries, 0 to 20590
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DATE            20591 non-null  object 
 1   거래단위            20591 non-null  object 
 2   도매시장            20591 non-null  object 
 3   산지-광역시도         20591 non-null  object 
 4   등급              20591 non-null  object 
 5   단위_무게(kg)       20591 non-null  int64  
 6   1kg_물량          20591 non-null  int64  
 7   1kg_평균가격        20591 non-null  float64
 8   weighted_price  20591 non-null  float64
 9   광역시도            20591 non-null  object 
 10  평균기온            20591 non-null  float64
 11  평균풍속            20591 non-null  float64
 12  평균강수량           20591 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 2.0+ MB


# 데이터 부족한 도매시장제거

In [14]:
cabbage_df['DATE'] = pd.to_datetime(cabbage_df['DATE'])

# Month 열을 추가하여 날짜에서 월 정보 확인
cabbage_df['Month'] = cabbage_df['DATE'].dt.month

# 각 도매시장별 월별 데이터 개수를 확인
monthly_data_availability = cabbage_df.groupby(['도매시장', 'Month']).size().unstack(fill_value=0)

# 모든 월(1월~12월)에 데이터가 존재하는 도매시장만 필터링
sufficient_data_markets = monthly_data_availability[monthly_data_availability.min(axis=1) > 0].index

# 충분한 데이터를 가진 도매시장만 포함
filtered_cabbage_df = cabbage_df[cabbage_df['도매시장'].isin(sufficient_data_markets)]

# 신뢰구간으로 이상치 제거

In [15]:
# 80% 신뢰구간 (하위 10%, 상위 90%) 경계 계산
lower_bound = filtered_cabbage_df['1kg_평균가격'].quantile(0.10)
upper_bound = filtered_cabbage_df['1kg_평균가격'].quantile(0.90)

# 80% 신뢰구간 내의 데이터만 필터링
confidence_filtered_data_80 = filtered_cabbage_df[
    (filtered_cabbage_df['1kg_평균가격'] >= lower_bound) & 
    (filtered_cabbage_df['1kg_평균가격'] <= upper_bound)
]

# 90% 신뢰구간 (하위 5%, 상위 95%) 경계 계산
lower_bound_90 = filtered_cabbage_df['1kg_평균가격'].quantile(0.05)
upper_bound_90 = filtered_cabbage_df['1kg_평균가격'].quantile(0.95)

# 90% 신뢰구간 내의 데이터만 필터링
confidence_filtered_data_90 = filtered_cabbage_df[
    (filtered_cabbage_df['1kg_평균가격'] >= lower_bound_90) & 
    (filtered_cabbage_df['1kg_평균가격'] <= upper_bound_90)
]



In [ ]:
# filtered_cabbage_df.to_csv("data-files/filtered_cabbage_df.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# 재배 유형 정의
cultivation_periods = {
    "봄재배": {"planting": [3, 4, 5, 6], "harvesting": [6, 7]},
    "가을재배": {"planting": [8, 9, 10], "harvesting": [10, 11]},
    "월동재배": {"planting": [10, 11], "harvesting": [3, 4, 5]},
    "여름재배": {"planting": [6, 7, 8], "harvesting": [9, 10]}
}

# 각 재배 유형에 대해 파종 및 수확 여부를 나타내는 열 추가
for cultivation, periods in cultivation_periods.items():
    confidence_filtered_data_80[cultivation] = (
        confidence_filtered_data_80['Month'].isin(periods['planting'] + periods['harvesting'])
    ).astype(int)

# '도매시장' 열 원-핫 인코딩 수행
filtered_data_80 = pd.get_dummies(confidence_filtered_data_80, columns=['도매시장'], prefix='Market')

# 피처(feature)와 타겟(target) 선택
features = [
    '1kg_물량', '평균기온', '평균풍속', '평균강수량'
] + list(cultivation_periods.keys()) + [col for col in filtered_data_80.columns if col.startswith('Market_')]
X = filtered_data_80[features]
y = filtered_data_80['1kg_평균가격']

# 결측값 처리: 결측값이 포함된 행 제거
X = X.dropna()
y = y.loc[X.index]

# 피처 스케일링: 데이터 정규화를 위해 StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



C:\Users\human\AppData\Local\Temp\ipykernel_9960\3000730089.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confidence_filtered_data_80[cultivation] = (
C:\Users\human\AppData\Local\Temp\ipykernel_9960\3000730089.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confidence_filtered_data_80[cultivation] = (
C:\Users\human\AppData\Local\Temp\ipykernel_9960\3000730089.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

# 랜덤 포레스트

In [20]:
# 랜덤 포레스트 모델 초기화 및 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = rf_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(129.96207687660143),
 np.float64(28760.73994951413),
 np.float64(169.58991700426688),
 0.38955407069129455)

In [21]:
target_mean = y.mean()

mae_ratio = mae / target_mean

target_mean, mae_ratio

(np.float64(639.6492956891403), np.float64(0.20317708117943584))

In [ ]:
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 하이퍼파라미터 설정
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [None, 10, 20, 30],  # 최대 깊이
    'min_samples_split': [2, 5, 10],  # 노드 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드의 최소 샘플 수
}

# 랜덤 포레스트 모델 초기화
rf_model = RandomForestRegressor(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',  # MAE를 최소화
    cv=3,  # 3-폴드 교차검증
    verbose=2,
    n_jobs=-1  # 병렬 처리
)

# 하이퍼파라미터 튜닝 실행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 및 성능 확인
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # MAE는 음수로 계산되므로 부호 변환

best_params, best_score


Fitting 3 folds for each of 108 candidates, totalling 324 fits


({'max_depth': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 10,
  'n_estimators': 200},
 np.float64(134.51359059943948))

# XGBoost

In [26]:
from xgboost import XGBRegressor

# 랜덤 포레스트 모델 초기화 및 학습
xgb_model = XGBRegressor(random_state=42, objective='reg:squarederror')
xgb_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = xgb_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(130.94935548748737),
 np.float64(28401.583372727073),
 np.float64(168.52769319232692),
 0.39717715934162023)

# 로그

In [32]:
# 1차: 월별, 도매시장별, 산지-광역시도별로 IQR을 이용한 이상치 제거
def remove_outliers_iqr(group, column):
    Q1 = group[column].quantile(0.25)
    Q3 = group[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group[column] >= lower_bound) & (group[column] <= upper_bound)]

# 그룹화하여 1차 이상치 제거
filtered_cabbage_df_no_outliers_1 = filtered_cabbage_df.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='1kg_평균가격')

# 2차: 로그 변환 적용 후 이상치 제거
filtered_cabbage_df_no_outliers_1['log_1kg_평균가격'] = np.log1p(filtered_cabbage_df_no_outliers_1['1kg_평균가격'])  # 로그 변환

filtered_cabbage_df_log = filtered_cabbage_df_no_outliers_1.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='log_1kg_평균가격')



C:\Users\human\AppData\Local\Temp\ipykernel_9960\3434969813.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_cabbage_df_no_outliers_1 = filtered_cabbage_df.groupby(['Month', '도매시장', '산지-광역시도'], group_keys=False).apply(remove_outliers_iqr, column='1kg_평균가격')
C:\Users\human\AppData\Local\Temp\ipykernel_9960\3434969813.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_cabbage_df_log = 

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 재배 유형 정의
cultivation_periods = {
    "봄재배": {"planting": [3, 4, 5, 6], "harvesting": [6, 7]},
    "가을재배": {"planting": [9, 10], "harvesting": [10, 11]},
    "월동재배": {"planting": [9, 10, 11], "harvesting": [3, 4, 5]},
    "여름재배": {"planting": [6, 7, 8], "harvesting": [9, 10]}
}

# 각 재배 유형에 대해 파종 및 수확 여부를 나타내는 열 추가
for cultivation, periods in cultivation_periods.items():
    filtered_cabbage_df_log[cultivation] = (
        filtered_cabbage_df_log['Month'].isin(periods['planting'] + periods['harvesting'])
    ).astype(int)

# '도매시장' 열 원-핫 인코딩 수행
filtered_data_80 = pd.get_dummies(filtered_cabbage_df_log, columns=['도매시장'], prefix='Market')

# 피처(feature)와 타겟(target) 선택
features = [
    '1kg_물량', '평균기온', '평균풍속', '평균강수량'
] + list(cultivation_periods.keys()) + [col for col in filtered_data_80.columns if col.startswith('Market_')]
X = filtered_data_80[features]
y = filtered_data_80['1kg_평균가격']

# 결측값 처리: 결측값이 포함된 행 제거
X = X.dropna()
y = y.loc[X.index]

# 피처 스케일링: 데이터 정규화를 위해 StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [38]:
# 랜덤 포레스트 모델 초기화 및 학습
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = rf_model.predict(X_test)

# 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 평가 결과 출력
mae, mse, rmse, r2

(np.float64(170.97903660002737),
 np.float64(65113.221925301885),
 np.float64(255.17292553345445),
 0.5232280694588152)

# 1 진행중

In [4]:
# 작형별 파종기와 정식기 정의
data = pd.read_csv("data-files/absolute_filtered_data.csv")

cultivation_periods = {
    "봄재배": {"파종기": [3, 4], "정식기": [4, 5]},
    "여름재배": {"파종기": [5, 6], "정식기": [6, 7]},
    "가을재배": {"파종기": [7], "정식기": [8]},
    "월동재배": {"파종기": [9], "정식기": [10]}
}

# 특정 기간의 기후 데이터(평균 기온)를 계산하는 함수
def calculate_period_temperature(data, months):
    period_data = data[data['Month'].isin(months)]
    return period_data['평균기온'].mean()

# 작형별 파종기와 정식기의 평균 기온 계산
features = []
for cultivation, periods in cultivation_periods.items():
    # 파종기 평균 기온 계산
    sowing_temp = calculate_period_temperature(data, periods['파종기'])
    # 정식기 평균 기온 계산
    transplanting_temp = calculate_period_temperature(data, periods['정식기'])
    
    # 결과 저장
    features.append({
        '작형': cultivation,
        '파종기_평균기온': sowing_temp,
        '정식기_평균기온': transplanting_temp
    })

# 데이터프레임으로 변환
features_df = pd.DataFrame(features)

In [5]:
# '작형' 열을 생성하여 재배 방식 할당
def assign_cultivation_type(row, cultivation_periods):
    for cultivation, periods in cultivation_periods.items():
        if row['Month'] in periods['파종기'] + periods['정식기']:
            return cultivation
    return None

# '작형' 열 추가
data['작형'] = data.apply(assign_cultivation_type, axis=1, cultivation_periods=cultivation_periods)

# '작형' 열을 기반으로 데이터 병합
data_with_features = pd.merge(data, features_df, on='작형', how='left')

In [10]:
data_with_features

,DATE,거래단위,도매시장,산지-광역시도,등급,단위_무게(kg),1kg_물량,1kg_평균가격,weighted_price,광역시도,...,평균풍속,평균강수량,Month,group_mean,group_std,lower_bound,upper_bound,작형,파종기_평균기온,정식기_평균기온
0,2024-10-01,8kg그물망,서울가락도매,강원도,특,8,22824192,1132.094998,2.583915e+10,강원도,...,1.56,130.36,10,940.639871,409.993040,-289.339249,2170.618991,월동재배,20.112654,13.270599
1,2024-10-01,8kg그물망,서울가락도매,강원도,특,8,10340416,1248.230942,1.290723e+10,강원도,...,1.56,130.36,10,940.639871,409.993040,-289.339249,2170.618991,월동재배,20.112654,13.270599
2,2024-10-01,8kg그물망,서울가락도매,강원도,특,8,6676480,828.431629,5.531007e+09,강원도,...,1.56,130.36,10,940.639871,409.993040,-289.339249,2170.618991,월동재배,20.112654,13.270599
3,2024-10-01,8kg그물망,서울가락도매,강원도,특,8,6619776,990.455145,6.556591e+09,강원도,...,1.56,130.36,10,940.639871,409.993040,-289.339249,2170.618991,월동재배,20.112654,13.270599
4,2024-10-01,8kg그물망,서울가락도매,강원도,특,8,5156480,984.768214,5.077938e+09,강원도,...,1.56,130.36,10,940.639871,409.993040,-289.339249,2170.618991,월동재배,20.112654,13.270599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20037,2023-06-01,12kg상자,울산도매시장,강원도,특,12,144,791.666667,1.140000e+05,강원도,...,1.79,213.89,6,875.000000,117.851130,521.446609,1228.553391,여름재배,19.689843,23.339829
20038,2023-09-01,12kg상자,부산반여도매,광주광역시,특,12,7200,416.666667,3.000000e+06,광주광역시,...,1.48,151.00,9,481.666667,91.923882,205.895022,757.438311,월동재배,20.112654,13.270599
20039,2022-09-01,15kg상자,부산반여도매,광주광역시,특,15,2700,546.666667,1.476000e+06,광주광역시,...,2.16,95.06,9,481.666667,91.923882,205.895022,757.438311,월동재배,20.112654,13.270599
20040,2024-10-01,15kg상자,서울강서도매,경상북도,특,15,201600,583.318452,1.175970e+08,경상북도,...,1.35,96.98,10,695.477792,158.617259,219.626016,1171.329568,월동재배,20.112654,13.270599


In [ ]:
def calculate_harvest_avg_price(data, harvest_periods):
    avg_prices = []
    for cultivation, months in harvest_periods.items():
        # 작형과 월 기준으로 수확기 데이터 필터링
        harvest_data = data[(data['작형'] == cultivation) & (data['Month'].isin(months))]
        avg_price = harvest_data['1kg_평균가격'].mean() if not harvest_data.empty else None
        avg_prices.append({'작형': cultivation, '수확기_1kg_평균가격': avg_price})
    return pd.DataFrame(avg_prices)

# 수확기 평균가격 계산
harvest_avg_prices = calculate_harvest_avg_price(data, harvest_periods)

# 원본 데이터에 수확기 평균가격 추가
final_data = pd.merge(data, harvest_avg_prices, on='작형', how='left')

NameError: name 'harvest_peridos' is not defined

In [19]:
data_with_features['작형'].isna

<bound method Series.isna of 0        월동재배
1        월동재배
2        월동재배
3        월동재배
4        월동재배
         ... 
20037    여름재배
20038    월동재배
20039    월동재배
20040    월동재배
20041    월동재배
Name: 작형, Length: 20042, dtype: object>